In [ ]:
pip install ultralytics

In [ ]:
#!yolo task=detect mode=train model=yolov5s.pt data=gesture_data.yaml epochs=30 imgsz=640
import subprocess

cmd = "yolo task=detect mode=train model=yolov5s.pt data=gesture_data.yaml epochs=30 imgsz=640"
process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

for line in iter(process.stdout.readline, b''):
    print(line.decode('utf-8').strip())



PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

New https://pypi.org/project/ultralytics/8.3.130 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.62 🚀 Python-3.12.4 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-12450H)
engine\trainer: task=detect, mode=train, model=yolov5s.pt, data=gesture_data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False

In [5]:
!yolo task=detect mode=val model=runs/detect/train5/weights/best.pt data=gesture_data.yaml


Ultralytics 8.3.62 ðŸš€ Python-3.12.4 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-12450H)


val: Scanning C:\Users\David\Desktop\model_yolo\dataset\labels\validation.cache... 139 images, 0 backgrounds, 0 corrupt: 100%|##########| 139/139 [00:00<?, ?it/s]
val: Scanning C:\Users\David\Desktop\model_yolo\dataset\labels\validation.cache... 139 images, 0 backgrounds, 0 corrupt: 100%|##########| 139/139 [00:00<?, ?it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/9 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  11%|#1        | 1/9 [00:03<00:26,  3.34s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  22%|##2       | 2/9 [00:07<00:24,  3.55s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  33%|###3      | 3/9 [00:10<00:20,  3.35s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  44%|####4     | 4/9 [00:12<00:15,  3.14s/it]



YOLOv5s summary (fused): 193 layers, 9,114,245 parameters, 0 gradients, 23.8 GFLOPs
                   all        139        139      0.991      0.998      0.995      0.792
                stanga         40         40      0.997          1      0.995      0.756
               dreapta         36         36      0.997          1      0.995      0.832
                   sus         24         24       0.99          1      0.995      0.719
                   jos         34         34          1      0.991      0.995      0.771
                  stop          5          5      0.971          1      0.995      0.881
Speed: 1.6ms preprocess, 169.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\val3
ðŸ’¡ Learn more at https://docs.ultralytics.com/modes/val


In [3]:
from ultralytics import YOLO
import cv2

# Încarcă modelul YOLO antrenat pentru gesturi
model = YOLO('runs/detect/train5/weights/best.pt')  # Înlocuiește cu calea corectă
gestures = ['stanga', 'dreapta', 'sus', 'jos', 'stop']  # Clasele gesturilor

# Mapare între gesturi și comenzi de zbor
commands = {
    'stanga': 'Motoare stânga',
    'dreapta': 'Motoare dreapta',
    'sus': 'Motoare sus',
    'jos': 'Motoare jos',
    'stop': 'Oprire motoare'
}

cap = cv2.VideoCapture(0)
cv2.namedWindow("Gesture Recognition", cv2.WINDOW_NORMAL)

scale_factor = 1
valo = None  # Gestul anterior

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Eroare: nu s-a putut captura cadrul.")
        break

    frame = cv2.flip(frame, 0)  # Oglindire orizontală
    results = model(frame)

    for result in results[0].boxes:
        class_id = int(result.cls.item())
        class_name = model.names[class_id]  # Numele clasei detectate

        if class_name in gestures:
            if class_name == valo:
                print("Comanda deja executată, selectează altă comandă.")
            else:
                # Afișează comanda de zbor asociată gestului
                print(f"Comandă nouă detectată: {commands[class_name]}")
                valo = class_name  # Actualizează ultimul gest executat

            # Desenează bounding box
            x1, y1, x2, y2 = map(int, result.xyxy[0].cpu().numpy())
            box_color = (0, 255, 0)
            box_thickness = max(2, int(frame.shape[0] * 0.005))
            cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, box_thickness)

            # Afișează numele gestului
            font_scale = max(0.5, frame.shape[0] / 1000)
            text_color = (0, 255, 0)
            cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, 2)

    resized_frame = cv2.resize(frame, (0, 0), fx=scale_factor, fy=scale_factor)
    cv2.imshow("Gesture Recognition", resized_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 6.0ms
Speed: 2.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 6.0ms
Speed: 2.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 7.0ms
Speed: 1.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 6.0ms
Speed: 1.9ms preprocess, 6.0ms inference, 1.0ms 

In [1]:
from ultralytics import YOLO
import cv2
import serial
import time

# Încarcă modelul YOLO antrenat pentru recunoaștere gesturi
model = YOLO('runs/detect/train5/weights/best.pt')  # Schimbă calea cu modelul tău
gestures = ['stanga', 'dreapta', 'sus', 'jos', 'stop']  # Clasele definite

# Inițializare conexiune serială către microcontroller
ser = serial.Serial('/dev/ttyUSB0', 9600, timeout=1)  # Ajustează portul și baud rate-ul
time.sleep(2)  # Așteaptă inițializarea conexiunii

# Pornire cameră
cap = cv2.VideoCapture(0)
cv2.namedWindow("Gesture Recognition", cv2.WINDOW_NORMAL)

valo = None  # Ultima comandă trimisă

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Eroare la capturarea imaginii.")
        break

    frame = cv2.flip(frame, 1)  # Oglindire pentru efect de oglindă

    results = model(frame)  # Inferență cu YOLO

    for result in results[0].boxes:
        class_id = int(result.cls.item())
        class_name = model.names[class_id]

        if class_name in gestures:
            if class_name != valo:
                valo = class_name
                print(f"Comandă nouă: {class_name}")
                ser.write((class_name + '\n').encode())  # Trimite comanda prin serial
            else:
                print("Aceeași comandă – nu se trimite din nou.")

            # Desenează cutia și eticheta
            x1, y1, x2, y2 = map(int, result.xyxy[0].cpu().numpy())
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, class_name, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    cv2.imshow("Gesture Recognition", cv2.resize(frame, (0, 0), fx=1, fy=1))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
ser.close()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'serial'